# Importing the necessary libraries

In [1]:
import yaml
import os
from pyspark.sql import SparkSession, Row
from pyspark.sql.types import StructType, StructField, StringType
import json

# Transforming the drivers table:

## getting all possible keys in all yaml files

In [19]:
def extractInfo(year, dataYaml, entrant):
    entryDrivers = []
    if 'constructorId' in dataYaml.keys():
        constructor = dataYaml['constructorId']
        entryDrivers.extend(drivers(year, dataYaml, entrant, constructor))
    else:
        for constructor in dataYaml['constructors']:
            entryDrivers.extend(drivers(year, constructor, entrant, constructor['constructorId']))
    return entryDrivers

def collectDriverInfo(year, entrant, constructor, dataYaml):
    data = {
            'year': int(year),
            'entrant' : entrant,
            'constructor' : constructor,
            'driver' : dataYaml['driverId'],
            'tookPart': False if dataYaml['rounds'] == None else True,
            'TestDriver' : dataYaml.get('testDriver', False)
        }
    return data

def drivers(year, dataYaml, entrant, constructor):
    driversList = []
    if 'driverId' in dataYaml.keys():
        driversList.append(collectDriverInfo(year, entrant, constructor, dataYaml))
    else:
        for driver in dataYaml['drivers']:
            driversList.append(collectDriverInfo(year, entrant, constructor, driver))
    return driversList


def getAllDrivers(folderPath = "/home/floppabox/f1/f1db/src/data/seasons"):
    allDrivers = []
    years = [year for year in os.listdir(folderPath)]
    for year in sorted(years, key=int):
        file_path = os.path.join(folderPath, year, 'entrants.yml')
        with open(file_path, 'r') as file:
            data=yaml.safe_load(file)
            for entry in data:
                allDrivers.extend(extractInfo(year, entry, entry['entrantId']))
    return allDrivers

print(getAllDrivers())


[{'year': 1950, 'entrant': 'alfa-romeo-spa', 'constructor': 'alfa-romeo', 'driver': 'juan-manuel-fangio', 'tookPart': True, 'TestDriver': False}, {'year': 1950, 'entrant': 'alfa-romeo-spa', 'constructor': 'alfa-romeo', 'driver': 'luigi-fagioli', 'tookPart': True, 'TestDriver': False}, {'year': 1950, 'entrant': 'alfa-romeo-spa', 'constructor': 'alfa-romeo', 'driver': 'nino-farina', 'tookPart': True, 'TestDriver': False}, {'year': 1950, 'entrant': 'alfa-romeo-spa', 'constructor': 'alfa-romeo', 'driver': 'reg-parnell', 'tookPart': True, 'TestDriver': False}, {'year': 1950, 'entrant': 'alfa-romeo-spa', 'constructor': 'alfa-romeo', 'driver': 'consalvo-sanesi', 'tookPart': True, 'TestDriver': False}, {'year': 1950, 'entrant': 'alfa-romeo-spa', 'constructor': 'alfa-romeo', 'driver': 'piero-taruffi', 'tookPart': True, 'TestDriver': False}, {'year': 1950, 'entrant': 'andy-granatelli', 'constructor': 'kurtis-kraft', 'driver': 'pat-flaherty', 'tookPart': True, 'TestDriver': False}, {'year': 1950,

In [ ]:
folderPath = "/home/floppabox/f1/f1db/src/data/seasons"

years = [year for year in os.listdir(folderPath)]
keysSeen = {}

for fileName in fileNames:
    file_path = os.path.join(folderPath, fileName)
    with open(file_path, 'r') as file:
        data=yaml.safe_load(file)
        for key in data.keys():
            if key not in keysSeen:
                keysSeen[key]=None
print(keysSeen)


In [20]:
spark = SparkSession.builder.appName("YAML to CSV").getOrCreate()


24/12/30 20:42:31 WARN Utils: Your hostname, Obuntu resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
24/12/30 20:42:31 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/30 20:42:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
from pyspark.sql.types import StringType, StructType, StructField, IntegerType, BooleanType

schema = StructType([
    StructField("year", IntegerType(), True),
    StructField("entrantId", StringType(), True),
    StructField("constructorId", StringType(), True),
    StructField("driverId", StringType(), True),
    StructField("tookPart", BooleanType(), True),
    StructField("testDriver", BooleanType(), True)
])
#year,entrant,constructor,driver,tookPart,TestDriver

readDataset = spark.read.schema(schema).option("header", True).csv("/home/floppabox/f1/f1-data-project-gr/csv_datasets/driversAllYears")


In [61]:

readDataset.createOrReplaceGlobalTempView("driversAllYears")

spark.sql("""SELECT MIN(year) as debut_year, driver
          FROM global_temp.driversAllYears
          WHERE tookPart == True
          GROUP BY driver""").show()

+----------+--------------------+
|debut_year|              driver|
+----------+--------------------+
|      2015|       roberto-merhi|
|      1999|    pedro-de-la-rosa|
|      2011|    jerome-dambrosio|
|      2003|       nicolas-kiesa|
|      2012|         charles-pic|
|      1996|giancarlo-fisichella|
|      2008|    nelson-piquet-jr|
|      2006|            yuji-ide|
|      2021|      nikita-mazepin|
|      2007|   markus-winkelhock|
|      1989|      johnny-herbert|
|      2009|     kamui-kobayashi|
|      2017|        pierre-gasly|
|      2014|        daniil-kvyat|
|      2001|      kimi-raikkonen|
|      2002|         mark-webber|
|      2011|    daniel-ricciardo|
|      2023|      logan-sargeant|
|      2014|        will-stevens|
|      2003|        ralph-firman|
+----------+--------------------+
only showing top 20 rows



In [70]:
spark.sql("""SELECT *
          FROM (
            SELECT MAX(year) as retirement_year, driver
            FROM global_temp.driversAllYears
            WHERE tookPart == True
            GROUP BY driver
          ) AS temp_ret
          WHERE retirement_year < (SELECT MAX(year) as maximum FROM global_temp.driversAllYears)""").show(150)

+---------------+--------------------+
|retirement_year|              driver|
+---------------+--------------------+
|           2015|       roberto-merhi|
|           2012|    pedro-de-la-rosa|
|           2012|    jerome-dambrosio|
|           2003|       nicolas-kiesa|
|           2013|         charles-pic|
|           2009|giancarlo-fisichella|
|           2009|    nelson-piquet-jr|
|           2006|            yuji-ide|
|           2021|      nikita-mazepin|
|           2007|   markus-winkelhock|
|           2000|      johnny-herbert|
|           2014|     kamui-kobayashi|
|           2020|        daniil-kvyat|
|           2021|      kimi-raikkonen|
|           2013|         mark-webber|
|           2015|        will-stevens|
|           2003|        ralph-firman|
|           2018|   stoffel-vandoorne|
|           2006|  juan-pablo-montoya|
|           2013|   heikki-kovalainen|
|           2018|     marcus-ericsson|
|           2014|    jean-eric-vergne|
|           2002|        

In [ ]:
drivers_data, drivers_relationships_data = [], []

# Extracting all drivers data

for file_name in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file_name)

    with open(file_path, 'r') as file:
        content_drivers=yaml.safe_load(file)
        record ={}

        for key in keys:
            if key != 'familyRelationships':
                record[key]= content_drivers.get(key)

        print(record)
        drivers_data.append(record)


In [ ]:

id=0
for file_name in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file_name)
    with open(file_path, 'r') as file:
        content_drivers=yaml.safe_load(file)
        
        if 'familyRelationships' in content_drivers.keys():
            for rel in content_drivers['familyRelationships']:
                record ={}
                record['id']=id
                id=id+1
                record['driverId']=content_drivers.get('id')
                record['relationId']=rel.get('driverId')
                record['type']=rel.get('type')
                drivers_relationships_data.append(record)




In [ ]:
real_keys= [x for x in keys if x != 'familyRelationships']

drivers= spark.createDataFrame(drivers_data).select(real_keys)

drivers_relationships = spark.createDataFrame(drivers_relationships_data).select(['id','driverId', 'relationId', 'type'])

drivers_relationships.show()